In [1]:
from multiprocessing import Pool
from multiprocessing import cpu_count
from anomalies import anomaly
import numpy as np
import pickle
from itertools import permutations 
from itertools import combinations_with_replacement
import pandas as pd

# Functions

In [2]:
def m_value(n):
    """
    m value for both even and odd cases
    
    Parameters
    ----------
    `n`: int, number of elements in the solution
    
    Returns
    -------
    `m`: int, number of elements in l and k lists
    """
    if n%2==0:
        m = n/2 -1
    else:
        m = (n-3)/2
    return int(m)

In [3]:
def all_comb_array(size):
    """
    A partir de un arreglo con los números [-5, -4, -3, -2, -1,  1,  2,  3,  4,  5]
    se crean arreglos de tamaño `size` que contengan todas las combinaciones posibles
    
    Parameters
    ----------
    `size`: int, tamaño de los arreglos que se quieren crear con las combinaciones de los números
            entre -5 y 5, sin considerar el cero
    
    Returns
    -------
    `final_combinations`: list, lista de tuplas de listas con todas las combinaciones de 
                        tamaño `size`
    """
    d_max = 15
    lista = np.arange(-d_max, d_max+1)
    ##Borramos el número cero de la lista
    lista_eff = np.delete(lista, 5)

    ##Creamos un arreglo con todas las combinaciones con reemplazo de tamaño dos con los objetos de lista
    ##Si yo tengo el arreglo [1,2,3] la combinación con reemplazo me da la siguiente solución
    ## A = [(1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (3, 3)]
    all_combinations = list(combinations_with_replacement(lista, size))

    ##La permutación de las anteriores combinaciones de A dan el siguiente resultado
    ##array([[[1, 1],[1, 1]],  ----> Eliminamos luego uno de estos
    ##       [[1, 2],[2, 1]],
    ##       [[1, 3],[3, 1]],
    ##       [[2, 2],[2, 2]],  ----> Eliminamos luego uno de estos
    ##       [[2, 3],[3, 2]],
    ##       [[3, 3],[3, 3]]]) ----> Eliminamos luego uno de estos
    perm = list(map(permutations, all_combinations))
    perm_list = np.array(list(map(list, perm)))
    shape = np.shape(perm_list)
    
    ##Volvemos todas las combinaciones en un arreglo 1D
    perm_list_1D = np.reshape(perm_list, (shape[0]*shape[1], shape[2]))
    ##Eliminamos los elementos repetidos
    final_combinations = [list(x) for x in set(tuple(x) for x in perm_list_1D)]
    return final_combinations


In [4]:
def l_and_k_arrays(n):
    """
    Genera arreglos para l y k dependiendo de `n` con los cuales se van a obtener las
    soluciones quirales
    
    Parameters
    ----------
    `n`: int, number of elements in the solution
    
    Returns
    -------
    `all_combinations_lk`: list, lista de tuplas con las posibles combinaciones de l y k [(l1,k1),..]
    """
    m = m_value(n)
    
    l_dummy = all_comb_array(m)
    
    ##Para el caso impar: l y k tienen dimensiones diferentes, los combinamos diferente :P
    if n %2 != 0:
        k_dummy = all_comb_array(m+1)
        #print(len(l_dummy), len(k_dummy))
        
        all_combinations_lk = [(i,j)for i in l_dummy for j in k_dummy]
    
    ##En el caso par: l y k tienen las mismas dimensiones
    else:
        all_combinations_lk = list(combinations_with_replacement(l_dummy, 2))
            
    return all_combinations_lk

x=l_and_k_arrays(5)

In [5]:
def is_vectorlike_solution(solution):
    """
    Verifica si al menos existe un elemento con su opuesto, si es el caso entonces es
    vectorlike y se marca como true para ser descartado
    
    Parameters
    ----------
    `solution`: list, solución a la operación merger
    
    Returns
    -------
    `isvectorlike`: bool, regresa verdadero si es una solución vectorlike
    """
    solution = np.array(solution)
    ##De la solución sacamos en valor absoluto los diferentes valores que hay
    values = np.unique(abs(solution))
    ##Asumimos de entrada que no es vectorlike
    isvectorlike = False
    
    
    for zabs in values:
        ##Si tenemos al menos una solución que contenga un cero ya decimos que es vectorlike
        if zabs == 0:
            isvectorlike = True
            break
            
        ##Miramos si para un valor está tanto su valor positivo como su negativo, si es así
        ##es vectorlike
        if - zabs in solution and zabs in solution:
            isvectorlike = True
            break
    return isvectorlike

In [6]:
def chiral_solution(n):
    """
    Soluciones quirales para `n` mayor o igual a 5
    
    Parameters
    ----------
    `n`: int, number of elements in the solution
    
    Returns
    -------
    `dict_sol`: list, lista de diccionarios con todas las soluciones quirales (incluye repetidas).
            Tiene la estructura: dict_sol = [{'n':int, 'l': list, 'k':list, 'z': list, 'gcd':int}]
    """
    
    vector = l_and_k_arrays(n)
    dict_sol = []
    
    for i in vector:
        #print(i[0], i[1])
    
        anomaly.free(i[0], i[1])
        solution = anomaly.free.simplified
        gcd = anomaly.free.gcd
        
        if solution[0] < 0:
            solution = -solution
        abs_sol = np.unique(abs(solution))
        ##Verificamos que sea una solución quiral y la guardamos
        if is_vectorlike_solution(solution) == False and np.all(abs_sol <= 32):
            dict_sol  += [{'n':n, 'l': i[0], 'k':i[1], 'z': solution, 'gcd':gcd}]
    
    return dict_sol

# Multiprocessing

In [8]:
n = int(input("¿Hasta que valor de n quiere calcular las soluciones quirales? mejor no se pase de 8"))
results = 0
processes = cpu_count() ##Para saber cuántos procesadores tengo 
use = 7  ##Tengo 8, entonces usaré 7
print(f"I have {processes} cores here. Using: {use}")
pool = Pool(processes=use)
results = pool.map(chiral_solution, np.arange(5,n+1)) ##Calculando los z quirales 
pool.close()
pool.join()

¿Hasta que valor de n quiere calcular las soluciones quirales? mejor no se pase de 8 7


I have 8 cores here. Using: 7


/home/lina/anaconda3/lib/python3.9/site-packages/anomalies/anomaly.py:49: RuntimeWarning: invalid value encountered in true_divide
  self.simplified=(zz/self.gcd).astype(int)
/home/lina/anaconda3/lib/python3.9/site-packages/anomalies/anomaly.py:49: RuntimeWarning: invalid value encountered in true_divide
  self.simplified=(zz/self.gcd).astype(int)
/home/lina/anaconda3/lib/python3.9/site-packages/anomalies/anomaly.py:49: RuntimeWarning: invalid value encountered in true_divide
  self.simplified=(zz/self.gcd).astype(int)


In [9]:
print(f'Las soluciones se han calculado para {len(results)} valores diferentes de n. El valor máximo es n = {n}')

Las soluciones se han calculado para 3 valores diferentes de n. El valor máximo es n = 7


# Quitando soluciones duplicadas

In [10]:
num_dict = len(results)   ##Número de n que se evaluaron
df = pd.DataFrame()

for dic in range(num_dict):
    df_dummy = pd.DataFrame(results[dic])
    df_dummy.sort_values('gcd', inplace=True)
    df_dummy['zs'] = df_dummy['z'].astype(str)
    df_dummy = df_dummy.drop_duplicates('zs').drop('zs',axis='columns').reset_index(drop=True)
    df = pd.concat([df, df_dummy],ignore_index=True)
df  ##dataframe con soluciones quirales únicas

,n,l,k,z,gcd
0,5,[1],"[-1, 3]","[2, 4, -7, -9, 10]",1
1,5,[-2],"[1, -4]","[4, 9, -14, -25, 26]",1
2,5,[-1],"[1, 5]","[2, 18, -23, -25, 28]",1
3,5,[1],"[-2, -1]","[1, 5, -7, -8, 9]",2
4,5,[1],"[-1, -6]","[1, 14, -17, -18, 20]",2
...,...,...,...,...,...
910,7,"[10, 4]","[-6, -8, 11]","[7, 11, -16, 18, -25, -26, 31]",52
911,7,"[9, 5]","[-9, 9, 12]","[8, 9, 19, -21, -21, -21, 27]",54
912,7,"[15, -9]","[-3, 8, -4]","[2, -4, 10, 10, -21, -27, 30]",108
913,7,"[7, 4]","[-5, -10, 15]","[3, -6, 7, 16, -25, -25, 30]",150


# Guardar los resultados

In [14]:
n = df['n'].max()
df.to_json(f'solution_{n}.json',orient='records')

# Número de soluciones por cada n

Se han considerado soluciones con enteros entre -32 y 32

In [15]:
Conteos = df.groupby(['n']).count()

Conteos

,l,k,z,gcd
n,,,,
5,12,12,12,12
6,142,142,142,142
7,761,761,761,761


# Soluciones del profesor

In [22]:
df_DR=pd.read_json('solutions_DR.json')

In [25]:
Conteos = df_DR.groupby(['n']).count()

Conteos

,l,k,solution,gcd
n,,,,
5,12,12,12,12
6,141,141,141,141
7,761,761,761,761
8,5569,5569,5569,5569
9,24882,24882,24882,24882
10,116864,116864,116864,116864
11,164981,164981,164981,164981
12,76864,76864,76864,76864


# Abrir los datos guardados

In [16]:
sol_loaded = pd.read_json('solution_7.json')

In [17]:
def eq_satisfied(x):
    """
    This function verifies that equations (z1³+z2³+ ... + zn³ = 0) and (z1+z2+ ... + zn = 0) are 
    satisfied at the same time, and if it is true, then return the input parameters
    
    Parameters
    ----------
    `x`: it correspond to the solutions z1, z2, ..., zn

    Return
    ------
    `eq_3`: int, returns the result of evaluating the equations
    `eq_1`: int, returns the result of evaluating the equations
    """
    x = np.array(x)
    eq_3 = np.sum(x**3)
    eq_1 = np.sum(x)

    return eq_3, eq_1

# Veamos si todas las soluciones obtenidas hasta $n=8$ son soluciones realmente

In [18]:
for sols in sol_loaded['z']:
    eq3, eq1 = eq_satisfied(sols)
    assert (eq3 ==0 and eq1 ==0)

In [19]:
print(len(sol_loaded))
sol_loaded

915


,n,l,k,z,gcd
0,5,[1],"[-1, 3]","[2, 4, -7, -9, 10]",1
1,5,[-2],"[1, -4]","[4, 9, -14, -25, 26]",1
2,5,[-1],"[1, 5]","[2, 18, -23, -25, 28]",1
3,5,[1],"[-2, -1]","[1, 5, -7, -8, 9]",2
4,5,[1],"[-1, -6]","[1, 14, -17, -18, 20]",2
...,...,...,...,...,...
910,7,"[10, 4]","[-6, -8, 11]","[7, 11, -16, 18, -25, -26, 31]",52
911,7,"[9, 5]","[-9, 9, 12]","[8, 9, 19, -21, -21, -21, 27]",54
912,7,"[15, -9]","[-3, 8, -4]","[2, -4, 10, 10, -21, -27, 30]",108
913,7,"[7, 4]","[-5, -10, 15]","[3, -6, 7, 16, -25, -25, 30]",150
